In [14]:
import pandas as pd
import numpy as np
import json

In [15]:
graph = "starwars"
embeddings_file = "results/embeddings/" + graph + "_lab_altlab_type_abs_comment_BAAI_bge-large-en-v1.5.json"
url_embeddings_file = "results/url_embeddings_short/" + graph + "_url_BAAI_bge-large-en-v1.5.json"
top_betweenness_file = "results/top_betweenness/" + graph + ".json"
community_embeddings_file = "results/community_embeddings/" + graph + ".json"

In [17]:
df = pd.read_json(embeddings_file)
df_T = df.T

In [18]:
# df_T

In [19]:
# df_T.loc[2239]

In [20]:
df_url = pd.read_json(url_embeddings_file)
df_url_T = df_url.T

In [21]:
# df_url_T

In [22]:
# df_url_T.loc[2239]

In [23]:
with open(top_betweenness_file, "r") as f:
    top_betweenness = json.load(f)

In [24]:
# top_betweenness[0]

In [25]:
# len(top_betweenness)

In [26]:
community_embeddings = []

for top_b in top_betweenness:

    embeddings = []

    top_b = {int(k): v for k, v in top_b.items()}
    keys = list(top_b.keys())
    weights = list(top_b.values())
    # print(keys)
    # print(weights)

    if sum(weights) != 0:
        top_b = {int(k): v for k, v in top_b.items() if v != 0}
        keys = list(top_b.keys())
        weights = list(top_b.values())

    for key in keys:
        try:
            selected_embedding = df_T.loc[key]
            embeddings.append(selected_embedding)
        except KeyError:
            try:
                selected_embedding = df_url_T.loc[key]
                embeddings.append(selected_embedding)
            except KeyError:
                top_b.pop(key)

    # keys = list(top_b.keys())
    # weights = list(top_b.values())
    # if sum(weights) == 0:
    #     weighted_avg = np.average(embeddings, axis=0)
    # else:
    #     weighted_avg = np.average(embeddings, axis=0, weights=weights)

    weighted_avg = np.average(embeddings, axis=0)

    # print(weighted_avg)
    community_embeddings.append(weighted_avg)

# print(community_embeddings)

embeddings_dict = {
    str(i): {str(j): val for j, val in enumerate(embedding)}
    for i, embedding in enumerate(community_embeddings)
}

with open(community_embeddings_file, "w") as json_file:
    json.dump(embeddings_dict, json_file, indent=4)